In [1]:
from numpy import mean
from numpy import std
from matplotlib import pyplot
from sklearn.model_selection import KFold
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
def load_dataset():
    # load dataset
    (trainX, trainY), (testX, testY) = mnist.load_data()
    # reshape dataset to have a single channel
    trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
    testX = testX.reshape((testX.shape[0], 28, 28, 1))
    # one hot encode target values
    trainY = to_categorical(trainY)
    testY = to_categorical(testY)
    return trainX, trainY, testX, testY

In [4]:
trainX, trainY, testX, testY = load_dataset()

In [5]:
trainX[0].shape

(28, 28, 1)

In [6]:
testX[0].shape

(28, 28, 1)

In [7]:
trainY.shape

(60000, 10)

In [8]:
def prep_pixels(train, test):
    # convert from integers to floats
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    # normalize to range 0-1
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    # return normalized images
    return train_norm, test_norm

In [9]:
def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(10, activation='softmax'))
    # compile model
    opt = SGD(lr=0.01, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [10]:
def evaluate_model(dataX, dataY, n_folds=5):
    scores, histories = list(), list()
    # prepare cross validation
    kfold = KFold(n_folds, shuffle=True, random_state=1)
    # enumerate splits
    for train_ix, test_ix in kfold.split(dataX):
        # define model
        model = define_model()
        # select rows for train and test
        trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
        # fit model
        history = model.fit(trainX, trainY, epochs=10, batch_size=32, validation_data=(testX, testY), verbose=1)
        # evaluate model
        acc = model.evaluate(testX, testY, verbose=1)
        print('> %.3f' % (acc * 100.0))
        # stores scores
        scores.append(acc)
        histories.append(history)
    return scores, histories

In [11]:
def summarize_diagnostics(histories):
    for i in range(len(histories)):
        # plot loss
        pyplot.subplot(2, 1, 1)
        pyplot.title('Cross Entropy Loss')
        pyplot.plot(histories[i].history['loss'], color='blue', label='train')
        pyplot.plot(histories[i].history['val_loss'], color='orange', label='test')
        # plot accuracy
        pyplot.subplot(2, 1, 2)
        pyplot.title('Classification Accuracy')
        pyplot.plot(histories[i].history['accuracy'], color='blue', label='train')
        pyplot.plot(histories[i].history['val_accuracy'], color='orange', label='test')
    pyplot.show()

In [12]:
# load dataset
trainX, trainY, testX, testY = load_dataset()
# prepare pixel data
trainX, testX = prep_pixels(trainX, testX)
# evaluate model
scores, histories = evaluate_model(trainX, trainY)
# learning curves
summarize_diagnostics(histories)

Epoch 1/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.1717 - accuracy: 0.9474 - val_loss: 0.0829 - val_accuracy: 0.9762
Epoch 2/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0587 - accuracy: 0.9824 - val_loss: 0.0661 - val_accuracy: 0.9806
Epoch 3/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0373 - accuracy: 0.9888 - val_loss: 0.0601 - val_accuracy: 0.9813
Epoch 4/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0253 - accuracy: 0.9925 - val_loss: 0.0583 - val_accuracy: 0.9833
Epoch 5/10
1500/1500 [==============================] - 5s 4ms/step - loss: 0.0171 - accuracy: 0.9952 - val_loss: 0.0567 - val_accuracy: 0.9841
Epoch 6/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0126 - accuracy: 0.9967 - val_loss: 0.0588 - val_accuracy: 0.9841
Epoch 7/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0074 - accuracy: 0.9984 - val_loss: 0.0627 - val_accuracy:

TypeError: can't multiply sequence by non-int of type 'float'

In [ ]:
scores

In [ ]:
sum(histories[0].history['accuracy'])/len(histories[0].history['accuracy'])